<a href="https://colab.research.google.com/github/yerinsally/BDA/blob/main/%EC%A0%95%EC%98%88%EB%A6%B0_0326_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BDA 5기/data/df_scaled.csv')
df2 = pd.read_csv('/content/drive/MyDrive/[BDAA 5기] 데이터 분석 기초반/데이터셋/call_data.csv')

## 해지여부를 예측하는 모델링
- 이진분류를 하는 모델링
- LGBM, XGB, CAT, RF 등 다양하게 분류에대한 알고리즘을 사용 
- y 값은 전화해지여부
- x 값은 나머지

In [4]:
df1.columns

Index(['가입일', '음성사서함이용', '주간통화시간', '주간통화횟수', '주간통화요금', '저녁통화시간', '저녁통화횟수',
       '저녁통화요금', '밤통화시간', '밤통화횟수', '밤통화요금', '상담전화건수'],
      dtype='object')

In [5]:
df1

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수
0,1.367809,-0.482963,-1.793364,-0.651519,-0.595007,0.074973,-0.885282,0.432819,-0.007296,1.680621,2.211081,-0.140478
1,-1.274600,2.726676,0.868896,1.612898,1.838269,0.083074,1.639215,0.821967,0.804608,0.510741,2.150534,-0.140478
2,-0.539251,0.640411,0.377201,1.612898,-0.297319,-0.759444,1.600075,0.911582,0.235326,1.680621,-0.403081,-1.115749
3,0.511248,-0.442843,-0.345531,0.628369,-0.689922,-0.408395,-0.953776,-0.183435,-0.808357,0.756514,-0.288360,-1.115749
4,0.503168,-0.482963,-1.827724,0.618523,-0.534606,-0.531262,-0.875497,0.329701,-0.004585,0.648374,-0.985186,2.785335
...,...,...,...,...,...,...,...,...,...,...,...,...
30195,0.834479,2.726676,0.462508,0.411772,-0.832294,0.232945,1.609860,0.550668,-0.751428,-0.895080,-0.296858,-0.140478
30196,0.996094,2.766797,-0.472305,1.189550,2.131642,0.281552,0.425891,0.909127,0.330207,0.451756,-0.615528,-0.140478
30197,-1.096823,-0.482963,-0.333683,-1.242237,-0.819351,-0.396243,-1.237537,-0.389671,0.408822,-0.609983,-0.834349,-0.140478
30198,-0.781673,-0.442843,0.140240,0.421618,-0.495778,0.234295,1.208681,0.427909,0.236682,0.491080,0.935336,-0.140478


In [11]:
df_train = df1[['가입일', '음성사서함이용', '상담전화건수', '주간통화시간', '저녁통화시간', '밤통화시간',
       '주간통화횟수', '저녁통화횟수', '밤통화횟수', '주간통화요금', '저녁통화요금', '밤통화요금']]
df_y = df2['전화해지여부']

In [12]:
df_train

,가입일,음성사서함이용,상담전화건수,주간통화시간,저녁통화시간,밤통화시간,주간통화횟수,저녁통화횟수,밤통화횟수,주간통화요금,저녁통화요금,밤통화요금
0,1.367809,-0.482963,-0.140478,-1.793364,0.074973,-0.007296,-0.651519,-0.885282,1.680621,-0.595007,0.432819,2.211081
1,-1.274600,2.726676,-0.140478,0.868896,0.083074,0.804608,1.612898,1.639215,0.510741,1.838269,0.821967,2.150534
2,-0.539251,0.640411,-1.115749,0.377201,-0.759444,0.235326,1.612898,1.600075,1.680621,-0.297319,0.911582,-0.403081
3,0.511248,-0.442843,-1.115749,-0.345531,-0.408395,-0.808357,0.628369,-0.953776,0.756514,-0.689922,-0.183435,-0.288360
4,0.503168,-0.482963,2.785335,-1.827724,-0.531262,-0.004585,0.618523,-0.875497,0.648374,-0.534606,0.329701,-0.985186
...,...,...,...,...,...,...,...,...,...,...,...,...
30195,0.834479,2.726676,-0.140478,0.462508,0.232945,-0.751428,0.411772,1.609860,-0.895080,-0.832294,0.550668,-0.296858
30196,0.996094,2.766797,-0.140478,-0.472305,0.281552,0.330207,1.189550,0.425891,0.451756,2.131642,0.909127,-0.615528
30197,-1.096823,-0.482963,-0.140478,-0.333683,-0.396243,0.408822,-1.242237,-1.237537,-0.609983,-0.819351,-0.389671,-0.834349
30198,-0.781673,-0.442843,-0.140478,0.140240,0.234295,0.236682,0.421618,1.208681,0.491080,-0.495778,0.427909,0.935336


In [13]:
df_y

0        0
1        0
2        0
3        0
4        0
        ..
30195    0
30196    0
30197    0
30198    1
30199    0
Name: 전화해지여부, Length: 30200, dtype: int64

### 내 과제 모델링

- 데이터를 학습하기 위해서 데이터를 나눠서 학습 시켜야 한다.
- trian, test split 진행
- 교차검증 cross_validate 교차검증이라는 건 split 랜덤으로 나눠서 kfold, Skfold 

In [14]:
model = [DecisionTreeClassifier(), RandomForestClassifier(), LogisticRegression()]
name = ['DT','RF','LR']

for model, name in zip(model, name):
    print('########## 사용하는 알고리즘',name,'########')
    for score in ['accuracy','precision','recall','f1']:
        print(score)
        print('--')
        print(cross_val_score(model, df_train, df_y, scoring=score, cv=3 ))

########## 사용하는 알고리즘 DT ########
accuracy
--
[0.87881196 0.88576537 0.8800914 ]
precision
--
[0.45311131 0.48219178 0.44415584]
recall
--
[0.48101266 0.48462929 0.45388788]
f1
--
[0.46263345 0.47165533 0.45495495]
########## 사용하는 알고리즘 RF ########
accuracy
--
[0.90980431 0.91139366 0.91128552]
precision
--
[0.7047817  0.72469636 0.72834646]
recall
--
[0.31103074 0.32188065 0.32549729]
f1
--
[0.43001261 0.4395466  0.44554455]
########## 사용하는 알고리즘 LR ########
accuracy
--
[0.89013609 0.89013609 0.89012517]
precision
--


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0. 0. 0.]
recall
--
[0. 0. 0.]
f1
--
[0. 0. 0.]


In [17]:
print((0.43001261 + 0.4395466 + 0.44554455)/3)

0.43836791999999997


### 최종 정리
- 사용한 모델 중 Random Forest 성능이 가장 좋음
- F1-score = [0.43001261, 0.4395466,  0.44554455] = 0.43836791999999997
- recall score가 상대적으로 매우 낮음
  - 실제 True인 것 중에서 모델이 True라고 예측한 것의 비율
  - 재현율 낮은 이유가 뭘까...
